### install pymongo package
- mac
    - pip(3) install pymongo
- window
    - conda install -c anaconda pymongo

In [1]:
import pymongo, requests

##### 1. server에 연결(client생성)

In [2]:
client = pymongo.MongoClient('mongodb://13.125.237.246:27017')
client

MongoClient(host=['13.125.237.246:27017'], document_class=dict, tz_aware=False, connect=True)

##### 2. db선택

In [3]:
db = client.dss
db

Database(MongoClient(host=['13.125.237.246:27017'], document_class=dict, tz_aware=False, connect=True), 'dss')

##### 3. db의 collection 리스트를 확인

In [4]:
db.collection_names()

['user', 'info2', 'info1', 'info']

##### 4. collection 선택

In [5]:
collection = db.info
collection

Collection(Database(MongoClient(host=['13.125.237.246:27017'], document_class=dict, tz_aware=False, connect=True), 'dss'), 'info')

##### 5. find

In [8]:
# find_one : 한 개의 document를 가져옵니다.
document = collection.find_one({"subject" : "java"})
type(document), document

(dict,
 {'_id': ObjectId('5b5956e6485331f76a0f8eb2'),
  'subject': 'java',
  'level': 4.0,
  'comments': [{'name': 'po', 'msg': 'check'}]})

In [9]:
# find : 여러 개의 documents를 가져옵니다
documents = collection.find({"subject": "java"})
documents

In [10]:
datas = list(documents)
len(datas)

2

In [11]:
datas

[{'_id': ObjectId('5b5956e6485331f76a0f8eb2'),
  'subject': 'java',
  'level': 4.0,
  'comments': [{'name': 'po', 'msg': 'check'}]},
 {'_id': ObjectId('5b5956e6485331f76a0f8eb6'),
  'subject': 'java',
  'level': 4.0,
  'comments': [{'name': 'alice', 'msg': 'hello'}]}]

In [13]:
list(documents)

[]

다 사라짐.

In [14]:
# count - documents의 갯수를 가져오는 함수
documents = collection.find()
documents.count()

8

In [18]:
# sort - 정렬
documents = collection.find({"level":{"$lte":3}}).sort("level", pymongo.DESCENDING)
list(documents)

[{'_id': ObjectId('5b5956e6485331f76a0f8eb3'),
  'subject': 'javascript',
  'comments': [{'name': 'jin', 'msg': 'nice'}],
  'level': 3.0},
 {'_id': ObjectId('5b5956e6485331f76a0f8eb4'),
  'subject': 'html',
  'comments': [],
  'level': 3.0},
 {'_id': ObjectId('5b5956e6485331f76a0f8eb1'),
  'subject': 'r',
  'level': 1.0,
  'comments': [{'name': 'jin', 'msg': 'bad'}, {'name': 'alice', 'msg': 'hi'}]},
 {'_id': ObjectId('5b5956e6485331f76a0f8eb5'),
  'subject': 'sass',
  'level': 1.0,
  'comments': [{'name': 'alice', 'msg': 'hello'}]},
 {'_id': ObjectId('5b595a4de78ab03356d8bf27'),
  'subject': 'less',
  'level': 1.0,
  'comments': [{'name': 'alice', 'msg': 'hello'}]}]

##### 6. insert

In [20]:
# insert_one
data = {"subject":"css", "level":1, "comments":[{"name":"peter", "msg":"easy"}]}
result = collection.insert_one(data)
result

In [22]:
result.inserted_id

ObjectId('5b5967ea33868e3478b09182')

In [23]:
# insert_many
datas = [
     {"subject":"webpack", "level":2, "comments":[{"name":"peter", "msg":"easy"}]},
     {"subject":"gulp", "level":3, "comments":[{"name":"peter", "msg":"easy"}]},
     {"subject":"bower", "level":4, "comments":[{"name":"peter", "msg":"easy"}]}
]
result = collection.insert_many(datas)
result

In [24]:
result.inserted_ids

[ObjectId('5b59685e33868e3478b09183'),
 ObjectId('5b59685e33868e3478b09184'),
 ObjectId('5b59685e33868e3478b09185')]

### 직방 데이터 크롤링 후 저장

In [39]:
url = "https://api.zigbang.com/v3/items?detail=true&item_ids=[12258942,12217921,12251354,12042761,12270198,12263778,12149733,12263079,12046500,12227516,12245261,12258364,11741210,11947081,12081429,12248641,12039772,12148952,12271001,12201879,12269163,12268373,12268568,12204018,12247416,12241201,12174611,12254380,12233724,12139836,11869595,12178704,12262681,12261598,12106912,12248115,12154374,12240537,12245412,12155533,12198385,12203883,12251810,12239779,12013638,12218505,12249844,12184761,12258707,12096937,12191641,12256520,12163720,12241556,12245758,12272387,12256200,12260120,12195600,12263256]"

In [40]:
response = requests.get(url)
response

<Response [200]>

In [41]:
# parsing - [{},{},{},{},{},..........]
zigbang_dict_list = response.json().get("items") # 최상단 items를 벗겨냄
len(zigbang_dict_list)

60

In [42]:
items = [item["item"] for item in zigbang_dict_list]
len(items)

60

In [43]:
items[:2]

[{'id': 12258942,
  'images': [{'index': 0,
    'count': 1,
    'url': 'https://ic.zigbang.com/ic/users/1560562/photos/uploads/427b772f21a022151c65450b46408f973bdc6b30.2175880.JPG'},
   {'index': 1,
    'count': 2,
    'url': 'https://ic.zigbang.com/ic/users/1560562/photos/uploads/bc53e8df0795485d6dbbc0d9a5b4e9b2e2a8c623.2251266.JPG'},
   {'index': 2,
    'count': 3,
    'url': 'https://ic.zigbang.com/ic/users/1560562/photos/uploads/081448a6977a4636164a589d5c10a7e42d29adee.2673824.JPG'},
   {'index': 3,
    'count': 4,
    'url': 'https://ic.zigbang.com/ic/users/1560562/photos/uploads/c0eb587da049174004f35a968c3f99b294384d3c.2393596.JPG'},
   {'index': 4,
    'count': 5,
    'url': 'https://ic.zigbang.com/ic/users/1560562/photos/uploads/0dc8b5121c0c597de0cdf739ac395a5fe6897b3f.2791090.JPG'},
   {'index': 5,
    'count': 6,
    'url': 'https://ic.zigbang.com/ic/users/1560562/photos/uploads/d2e8fb5f4be6fe0bcaad67e1c222b3928d95e2c8.3091381.JPG'},
   {'index': 6,
    'count': 7,
    'url':

In [44]:
collection = client.crawling.zigbang
result_zigbang = collection.insert_many(items)
result_zigbang

##### 렌트비용이 50이하인 데이터 추출

In [45]:
query = {"rent":{"$lte":50}}

documents = collection.find(query)
documents

In [46]:
datas = list(documents)
len(datas)

56

In [47]:
# pandas로 만들어보자
df = pd.DataFrame(datas)
df.tail()

,_floor,_id,address1,address2,address3,agent_address1,agent_comment,agent_email,agent_lat,agent_lng,...,updated_at2,user_email,user_has_no_penalty,user_has_penalty,user_intro,user_mobile,user_name,user_no,user_phone,view_count
51,3층,5b596ef933868e3478b091f9,서울시 동작구 상도동,126-45,None,"서울특별시 동작구 상도로15길 112, 101호(상도동)","✔허위매물없는 1억원 책임보헙 가입 되있는 안전한 부동산\n\n✔ 전화,문자,카톡 ...",elf2493@hanmail.net,37.504440,126.936951,...,2018-07-26,jilstt@naver.com,True,False,"동작구 7호선, 1호선 풀옵션 원룸, 투룸 최다보유 우주부동산입니다. 합리적이고 알...",0507-1280-3389,중개보조원 최재열,597303,0507-1280-3389,5
52,3층,5b596ef933868e3478b091fa,서울시 관악구 봉천동,41-369,None,서울특별시 관악구 관악로17길 25,"반갑습니다.\n15년 전통의 예림공인중개사무소 임수진 과장입니다.\n2,7호선의 모...",jaesung0509@naver.com,37.480698,126.950256,...,2018-07-24,jaesung0509@naver.com,True,False,,0507-1280-4562,중개보조원 임수진,3712773,0507-1280-4562,72
53,3층,5b596ef933868e3478b091fb,서울시 관악구 봉천동,876-11,None,서울시 관악구 쑥고개로 45,24시문의가능.,yhcymj@naver.com,37.479528,126.941621,...,2018-07-25,yhcymj@naver.com,True,False,,0507-1281-3596,중개보조원 김대환,4103322,0507-1281-3596,82
54,1층,5b596ef933868e3478b091fc,서울시 관악구 봉천동,941-12,None,서울특별시 관악구 봉천동 933-20,,nadoyo2@naver.com,37.484135,126.940536,...,2018-07-17,nadoyo2@naver.com,True,False,,0507-1280-2740,중개보조원 윤명철,1434787,0507-1280-2740,219
55,2층,5b596ef933868e3478b091fd,서울시 관악구 봉천동,1603-18,None,서울특별시 관악구 남부순환로167길 14,\n☎허위매물 보고 시간낭비는 이제그만!\n☎전화 한통이면 끝!! 빠른문의 부탁드립...,nam76769470@gmail.com,37.484226,126.923531,...,2018-07-25,nam76769470@gmail.com,True,False,,0507-1280-5213,중개보조원 안진수,3383939,0507-1280-5213,135


In [34]:
filtered_df = df[['rent','options','size','deposit']]
filtered_df.tail()

,rent,options,size,deposit
51,0,"에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,옷장,신발장,싱크대",6.5,10000
52,0,"에어컨,냉장고,세탁기,인덕션,책상,책장,옷장,신발장,싱크대",8.0,10000
53,27,"에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,침대,옷장,신발장,싱크대",6.0,100
54,0,"에어컨,냉장고,세탁기,인덕션,신발장,싱크대",8.0,6500
55,26,"에어컨,냉장고,세탁기,가스레인지,전자레인지,책상,책장,침대,옷장,신발장,싱크대",5.0,100


In [35]:
query = {"rent":{"$lte":50}}

documents = collection.find(query, {"_id":False,"deposit":True, "rent":True, "options":True,"size":True})
documents

In [36]:
df = pd.DataFrame(list(documents))
df.tail()

,deposit,options,rent,size
51,10000,"에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,옷장,신발장,싱크대",0,6.5
52,10000,"에어컨,냉장고,세탁기,인덕션,책상,책장,옷장,신발장,싱크대",0,8.0
53,100,"에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,침대,옷장,신발장,싱크대",27,6.0
54,6500,"에어컨,냉장고,세탁기,인덕션,신발장,싱크대",0,8.0
55,100,"에어컨,냉장고,세탁기,가스레인지,전자레인지,책상,책장,침대,옷장,신발장,싱크대",26,5.0


##### delete - database

In [37]:
client.drop_database("crawling")

##### delete - collection

In [38]:
client.crawling.drop_collection("zigbang")

{'ok': 0.0, 'errmsg': 'ns not found', 'code': 26}